In [1]:
!nvidia-smi

Fri Aug 21 11:01:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!python -m pip install git+https://github.com/ShivamShrirao/dnn_from_scratch.git

  Cloning https://github.com/ShivamShrirao/dnn_from_scratch.git to /tmp/pip-req-build-g0_ouv17
  Running command git clone -q https://github.com/ShivamShrirao/dnn_from_scratch.git /tmp/pip-req-build-g0_ouv17
  Created wheel for dnn-from-scratch: filename=dnn_from_scratch-0.1.dev1-cp36-none-any.whl size=39847 sha256=98f22886475d0b737e7e2e3c77a5ffda396ddc05b97d6591edd7a5f948715d33
  Stored in directory: /tmp/pip-ephem-wheel-cache-wihd95oy/wheels/33/77/6c/1140f6295368e7045f3b49fdda98e7f07b2513b8b55093ca7b
Successfully built dnn-from-scratch


In [3]:
!git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

Cloning into 'deep_Q_learning_from_scratch'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 242 (delta 149), reused 168 (delta 75), pack-reused 0
Receiving objects: 100% (242/242), 196.18 KiB | 387.00 KiB/s, done.
Resolving deltas: 100% (149/149), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
%cd deep_Q_learning_from_scratch

/content/deep_Q_learning_from_scratch


In [2]:
!git pull origin master

From https://github.com/ShivamShrirao/deep_Q_learning_from_scratch
 * branch            master     -> FETCH_HEAD
Already up to date.


In [3]:
import gym
import time
import cv2
import numpy as np
from collections import deque

from settings import *
from agent import *
from experience import *

In [4]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=2e-6, target_update_thresh=1000)
D_exp = ReplayMemory(capacity=1_000_000, nlap=1)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 128)        relu             73856
__________________________________________________________________________________________
3 Flatten(Flatten)        (None, 12800)              echo             0
__________________________________________________________________________________________
4 Den

In [5]:
D_exp.current_state.nbytes/1024/1024/1024

5.9604644775390625

In [6]:
env = gym.make('Breakout-v4')       # v4 to ensure ball is fired when required

In [37]:
D_exp.len

1000000

In [ ]:
for i_episode in range(1200):
    obinit = env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    lives = 5
    fire = True
    start = time.time()
    for t in range(10_000):
        s_s = time.time()
#         env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        if fire:
            action = 1
            fire = False
        else:
            action = agt.get_action(state_que)
        next_observation, reward, done, info = env.step(action)
        if lives != info['ale.lives']:
            lives = info['ale.lives']
            reward = -1
            fire = True
        ep_score += reward
        if action==1:
            action = 0
        D_exp.store_transition(state, agt.actions.index(action), reward, done)
        observation = next_observation

        if (D_exp.len-D_exp.idx_len) > BATCH_SIZE:
            grads = agt.train(D_exp, BATCH_SIZE)
        print('\r', t, action, ep_score, end='  ')
        if done:
            break
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
    if not i_episode%8:
        agt.model.save_weights("/content/drive/My Drive/breakout.w8s")
env.close()

In [44]:
agt.epsilon = 0

In [27]:
from gym import wrappers

In [51]:
wrapped_env = wrappers.Monitor(env.env, '/content/videos/' + str(time.time()) + '/')
for i_episode in range(1):
    obinit = wrapped_env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    lives = 5
    fire = True
    preds = []
    reward_history = []
    start = time.time()
    t=-1
    while 1:
        t+=1
        s_s = time.time()
#         wrapped_env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        if fire:
            action = 1
            fire = False
        else:
            out = agt.predict(state_que)
            pidx = cp.argmax(out[0]).item()
            preds.append(out[0][pidx].item())
            action = agt.actions[pidx]
        next_observation, reward, done, info = wrapped_env.step(action)
        if lives != info['ale.lives']:
            lives = info['ale.lives']
            reward = -1
            fire = True
        ep_score += reward
        reward_history.append(reward)
        if action==1:
            action = 0
        observation = next_observation
        print('\r', t, ep_score, end='  ')
        if done:
            break
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
wrapped_env.close()

Episode 1 finished after 1708 timesteps, Score: 58.0, Epsilon: 0.000000, Time: 4.29


In [34]:
!rm /content/videos/* -rf